In [1]:
%cd ..

C:\Users\lcvri\projects_lc\lexisnexis_place_extraction


In [2]:
# standard library
import json
import pickle
from itertools import combinations

# third party
import pandas as pd
from rbo.rbo import rbo

# local
from src.config_ import PATHS, LEXISNEXIS
from src.doc_analysis import most_common

Load total and unique token and entity counts.

In [3]:
files = {
    'total': 'dct_total_tokens_and_entities.pkl',
    'unique': 'dct_unique_tokens_and_entities.pkl',
}

d = dict()
for key, file in files.items():
    with open(PATHS.results / file, 'rb') as f:
        d[key] = pickle.load(f)

Delete the alts from the data and add their count to the main key. 

In [4]:
with open(PATHS.parameters / 'alts_countries.json', 'r', encoding='utf8') as f:
    alts_countries = json.load(f)

for count_type in d:
    for batch in LEXISNEXIS.batches:
        for country in alts_countries:
            for alt in alts_countries[country]:
                n = d[count_type][batch]['countries'][alt]
                del d[count_type][batch]['countries'][alt]
                d[count_type][batch]['countries'][country] += n

## Unique entities per type and newspaper

In [5]:
geo_entities = ['countries', 'places', 'places_uk', 'places_nl', 'places_fr']

In [6]:
data = list()
for ent in geo_entities:
    for batch in d['total']:
        data.append([ent, batch, len(d['total'][batch][ent])])
pd.DataFrame(data, columns=['geo_entity', 'source', 'count']).set_index(['geo_entity', 'source']).unstack(0)

count                                     
geo_entity         countries places places_fr places_nl places_uk
source                                                           
leeuwarder_courant        52     61        11        32        19
telegraaf                 56     66         2        39        36
trouw                     82    119         1        41        49
volkskrant                86    153         2        57        72

## Number of occurrences per entity type and label
Each occurrence is counted.

In [7]:
df = pd.DataFrame()
for ent in geo_entities:
    df_ = most_common(d['total'], ent, n=12)
    df_ = pd.concat([df_], keys=[ent], names=['entity_type'])
    df = df.append(df_)
df.update(df.xs('count', level=1, axis=1, drop_level=False).astype('Int64'))
df

volkskrant                      trouw        \
                                   label count                label count   
entity_type ranking                                                         
countries   0                  Nederland   637  Verenigd Koninkrijk   502   
            1        Verenigd Koninkrijk   436            Nederland   494   
            2                  Frankrijk   336     Verenigde Staten   362   
            3                  Duitsland   261                China   224   
            4           Verenigde Staten   239            Duitsland   193   
            5                    Rusland   109              Rusland   155   
            6                      Polen    94            Frankrijk   153   
            7                      China    80              Turkije   102   
            8                    Ierland    68              Ierland    95   
            9                     Spanje    60               Spanje    71   
            10                   Turkije    52                Polen    63   
            11                 Hongarije    48                Japan    44   
places      0                    Brussel   404              Brussel   346   
            1                     Parijs   107               Parijs    59   
            2                    Berlijn    71              Berlijn    48   
            3                       Rome    46          Straatsburg    44   
            4                   New York    43                 Rome    42   
            5                     Dublin    37               Dublin    34   
            6                     Madrid    26               Moskou    32   
            7                     Moskou    19             New York    31   
            8                     Milaan    18               Madrid    29   
            9                 Kopenhagen    17            Barcelona    17   
            10                   Hamburg    16             Florence    16   
            11                     Wenen    15             Warschau    15   
places_uk   0                     Londen   465               Londen   317   
            1                 Manchester    41           Manchester    25   
            2                     Oxford    23               Whitby    13   
            3                    Bristol    23            Cambridge     8   
            4                 Birmingham    13                Dover     8   
            5                      Dover    12           Birmingham     7   
            6                  Liverpool    11              Belfast     7   
            7                   Brighton    11              Bristol     7   
            8                  Cambridge    10               Oxford     5   
            9                    Belfast     8            Edinburgh     4   
            10                 Edinburgh     8              Paisley     4   
            11                  Plymouth     8        West Bromwich     4   
places_nl   0                  Amsterdam   198            Amsterdam   110   
            1                   Den Haag    78             Den Haag    43   
            2                  Rotterdam    42            Rotterdam    23   
            3                    Utrecht    22            Groningen    21   
            4                 Amstelveen    17              Utrecht    16   
            5                 Maastricht    16              Tilburg    10   
            6                    Tilburg    15           Maastricht    10   
            7                  Groningen    15               Leiden     9   
            8                     Almere    13            Eindhoven     9   
            9                     Leiden    12           Wageningen     8   
            10                Vlissingen    10         Scheveningen     6   
            11                  Volendam     7                  Urk     5   
places_fr   0                   Drachten     1               Dokkum     2   
            1             

## Number of occurrences per entity type and label
Labels are only counted once per article.

In [8]:
df = pd.DataFrame()
for ent in geo_entities:
    df_ = most_common(d['unique'], ent, n=12)
    df_ = pd.concat([df_], keys=[ent], names=['entity_type'])
    df = df.append(df_)
df.update(df.xs('count', level=1, axis=1, drop_level=False).astype('Int64'))
df

volkskrant                      trouw        \
                                   label count                label count   
entity_type ranking                                                         
countries   0        Verenigd Koninkrijk   289  Verenigd Koninkrijk   291   
            1                  Nederland   210     Verenigde Staten   183   
            2           Verenigde Staten   159            Nederland   182   
            3                  Duitsland   122            Frankrijk    90   
            4                  Frankrijk   114            Duitsland    88   
            5                      Polen    51              Ierland    52   
            6                    Rusland    46              Rusland    46   
            7                      China    42                China    45   
            8                    Ierland    41                Polen    39   
            9                     Spanje    38               Spanje    38   
            10               Griekenland    37              Turkije    30   
            11                   Turkije    32                Japan    21   
places      0                    Brussel   191              Brussel   158   
            1                     Parijs    77               Parijs    42   
            2                    Berlijn    42              Berlijn    26   
            3                   New York    36               Dublin    24   
            4                     Dublin    24             New York    24   
            5                       Rome    17               Moskou    19   
            6                     Moskou    14                 Rome    17   
            7                      Wenen    14               Madrid    17   
            8                     Madrid    10          Straatsburg    13   
            9                   Warschau     8             Warschau    12   
            10                 Barcelona     8            Barcelona    11   
            11               Straatsburg     7                Wenen    11   
places_uk   0                     Londen   195               Londen   135   
            1                 Manchester    21           Manchester    15   
            2                     Oxford    14           Birmingham     6   
            3                  Liverpool    11            Cambridge     5   
            4                  Cambridge     8                Dover     5   
            5                    Belfast     7              Belfast     5   
            6                      Dover     7               Oxford     4   
            7                  Edinburgh     7            Edinburgh     3   
            8                    Bristol     7             Aberdeen     3   
            9                   Brighton     6               Whitby     3   
            10                   Paisley     4        West Bromwich     3   
            11                  Plymouth     4              Chelsea     2   
places_nl   0                  Amsterdam    77            Amsterdam    54   
            1                   Den Haag    42             Den Haag    30   
            2                    Utrecht    16            Rotterdam    16   
            3                  Rotterdam    14              Utrecht    13   
            4                     Leiden    12            Groningen     9   
            5                 Maastricht    10               Leiden     8   
            6                    Tilburg     8           Maastricht     7   
            7                  Groningen     7              Tilburg     6   
            8                  Eindhoven     5           Wageningen     6   
            9               Scheveningen     4            Eindhoven     5   
            10                Amersfoort     4               Arnhem     4   
            11                    Almere     4          Spijkenisse     3   
places_fr   0                   Drachten     1               Dokkum     1   
            1             

## Compare rankings between newspapers
First compare between entities:

In [9]:
results = pd.DataFrame()
for ent in geo_entities:
    rankings = df.xs(ent).xs('label', axis=1, level=1)
    for combo in combinations(rankings.columns, 2):
        data = rbo(rankings[combo[0]], rankings[combo[1]], 0.9)
        s = pd.Series(data=data, name=(ent, combo))
        if results.empty:
            results = s.to_frame()
        else:
            results = results.join(s, how='outer')
results = results.T
results.index = pd.MultiIndex.from_tuples(results.index)
results.index.names = ('geo_entity', 'comparison')

In [10]:
results.sort_values(by=['geo_entity', 'min'], ascending=False)

min       res       ext
geo_entity comparison                                                    
places_uk  (telegraaf, leeuwarder_courant)   0.660951  0.168216  0.714316
           (volkskrant, trouw)               0.650170  0.161465  0.712430
           (volkskrant, telegraaf)           0.594364  0.168216  0.647730
           (volkskrant, leeuwarder_courant)  0.583086  0.174041  0.627558
           (trouw, leeuwarder_courant)       0.545584  0.168216  0.598950
           (trouw, telegraaf)                0.510522  0.174041  0.554993
places_nl  (volkskrant, trouw)               0.768946  0.144495  0.848995
           (trouw, leeuwarder_courant)       0.726568  0.161465  0.788828
           (volkskrant, leeuwarder_courant)  0.714356  0.161465  0.776616
           (trouw, telegraaf)                0.710766  0.161465  0.773027
           (volkskrant, telegraaf)           0.691182  0.153624  0.762336
           (telegraaf, leeuwarder_courant)   0.613642  0.174041  0.658113
places_fr  (volkskrant, trouw)               0.155843  0.699157  0.450000
           (volkskrant, telegraaf)           0.155843  0.699157  0.450000
           (trouw, telegraaf)                0.155843  0.699157  0.450000
           (volkskrant, leeuwarder_courant)  0.000000  0.783000  0.000000
           (trouw, leeuwarder_courant)       0.000000  0.783000  0.000000
           (telegraaf, leeuwarder_courant)   0.000000  0.783000  0.000000
places     (volkskrant, trouw)               0.847396  0.106732  0.954128
           (volkskrant, leeuwarder_courant)  0.763832  0.144495  0.843881
           (trouw, leeuwarder_courant)       0.753990  0.144495  0.834039
           (volkskrant, telegraaf)           0.741741  0.161465  0.804001
           (trouw, telegraaf)                0.725502  0.161465  0.787762
           (telegraaf, leeuwarder_courant)   0.708343  0.161465  0.770603
countries  (telegraaf, leeuwarder_courant)   0.841321  0.121373  0.939158
           (volkskrant, telegraaf)           0.830011  0.121373  0.927848
           (volkskrant, leeuwarder_courant)  0.830011  0.121373  0.927848
           (trouw, leeuwarder_courant)       0.810624  0.106732  0.917356
           (volkskrant, trouw)               0.786205  0.121373  0.884042
           (trouw, telegraaf)                0.777548  0.121373  0.875385

Then compare when combining all counts into one:

In [28]:
results = pd.DataFrame()
rankings = df.xs('label', axis=1, level=1)
for combo in combinations(rankings.columns, 2):
    ranking1 = df.xs(combo[0], axis=1).sort_values('count', ascending=False).label
    ranking2 = df.xs(combo[1], axis=1).sort_values('count', ascending=False).label
    data = rbo(ranking1, ranking2, 0.8)
    s = pd.Series(data=data, name=combo)
    if results.empty:
        results = s.to_frame()
    else:
        results = results.join(s, how='outer')
results = results.T

In [29]:
results.sort_values('min', ascending=False)

min       res       ext
volkskrant leeuwarder_courant  0.925002  0.000002  0.925003
           telegraaf           0.910810  0.000002  0.910811
telegraaf  leeuwarder_courant  0.878022  0.000002  0.878023
trouw      leeuwarder_courant  0.824253  0.000002  0.824254
volkskrant trouw               0.813513  0.000002  0.813514
trouw      telegraaf           0.770114  0.000002  0.770115

## Number of lemma occurrences
All occurrences are counted.

In [30]:
most_common(d['total'], 'lemma', n=15)

volkskrant           trouw        telegraaf       leeuwarder_courant  \
             label count     label count      label count              label   
ranking                                                                        
0             gaan  1500        eu  1365     brexit   927             brexit   
1               eu  1376      gaan  1172       gaan   865               gaan   
2           brexit  1305    brexit  1154         eu   823               jaar   
3             jaar  1262      jaar  1139       jaar   754              komen   
4            komen  1242     groot  1053      komen   695              groot   
5            groot  1235     komen  1053      brits   670                 eu   
6         europees  1165      land  1033      groot   578                may   
7            maken   955  europees   855   europees   490              brits   
8           zeggen   935     brits   826       goed   473           europees   
9             land   865      zien   710       land   469             partij   
10           brits   817     maken   708  nederland   452              maken   
11        politiek   812       may   687      maken   447               land   
12            zien   805    europa   674        may   419               goed   
13            goed   785    zeggen   637     zeggen   410          nederland   
14             may   764      goed   621       zien   397             europa   

               
        count  
ranking        
0         590  
1         532  
2         516  
3         484  
4         482  
5         470  
6         442  
7         401  
8         391  
9         354  
10        332  
11        315  
12        311  
13        301  
14        299

## Number of unique lemma occurrences
Lemma's are only counted once per article.

In [31]:
most_common(d['unique'], 'lemma', n=15)

volkskrant          trouw       telegraaf       leeuwarder_courant  \
             label count    label count     label count              label   
ranking                                                                      
0           brexit   567   brexit   483    brexit   478             brexit   
1             gaan   451     gaan   380      gaan   326              groot   
2            komen   436    groot   377     komen   305               jaar   
3            groot   435    komen   368      jaar   295              komen   
4             jaar   400     jaar   351     groot   274               gaan   
5            maken   390    maken   313     brits   267              maken   
6            staan   351     land   300     maken   255              brits   
7             zien   336    brits   294      goed   235                 eu   
8             goed   335     goed   286        eu   227               goed   
9             land   331     zien   280      land   218              nieuw   
10           nieuw   322       eu   277      zien   208           europees   
11          zeggen   318    staan   268  europees   196               land   
12            twee   314   zeggen   266     nieuw   193              staan   
13           brits   305    nieuw   265    zeggen   193               zien   
14        politiek   304  krijgen   251     staan   193            premier   

               
        count  
ranking        
0         293  
1         216  
2         193  
3         193  
4         187  
5         171  
6         162  
7         151  
8         144  
9         143  
10        141  
11        132  
12        131  
13        129  
14        128